<a href="https://colab.research.google.com/github/atran13/MSc-Thesis-Signs-of-Gentrification/blob/main/Code/EasyOCR_Text_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install easyocr
import os,sys,shutil
import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
# import easyocr
# from paddleocr import PaddleOCR, draw_ocr

In [ ]:
# Specify language to identify from images
reader = easyocr.Reader(['en','nl'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

## Detect & crop

In [ ]:
def extract_text(origin, destination):
    """
    :origin: Folder containing images
    :destination: Folder to contain cropped-out text instances
    """
    no_text = 0
    num_file = 0
    instances = 0
    for file in glob.glob(os.path.join(origin,'*.jpg')):
        num_file += 1
        filename = os.path.split(file)[1] # {name}.jpg
        dir = os.path.join(destination, filename)

        # Read img
        img = cv2.imread(file)

        # Detect texts
        result = reader.detect(img, add_margin=0)

        # Crop & save text regions
        if not any([result[0][0], result[1][0]]): # if no text detected
            no_text += 1
        else:
            os.makedirs(dir) # make folder for image with text

            for i, box in enumerate(result[0][0]):
                instances += 1
                tl = (max(0,box[0]), max(0,box[2]))
                br = (max(0,box[1]), max(0,box[3]))
                cropped_rec = img[tl[1]:br[1], tl[0]:br[0]]

                des_path = os.path.join(dir, f'rec-{i}.jpg')
                if cropped_rec is not None:
                    cv2.imwrite(des_path, cropped_rec)
                else:
                    pass

            for i, box in enumerate(result[1][0]):
                instances += 1
                rect = cv2.minAreaRect(np.array(box, dtype=np.int32))

                width = int(rect[1][0])
                height = int(rect[1][1])

                bound = cv2.boxPoints(rect) ; bound = np.int0(bound)

                src_pts = bound.astype("float32")

                if width > height:
                    dst_pts = np.array([[0, height-1],
                                        [0, 0],
                                        [width-1, 0],
                                        [width-1, height-1]],
                                        dtype="float32")
                    perspective_matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
                    cropped_poly = cv2.warpPerspective(img, perspective_matrix, (width, height))
                else:
                    dst_pts = np.array([[0, 0],
                                        [height-1, 0],
                                        [height-1, width-1],
                                        [0, width-1]],
                                        dtype="float32")

                    perspective_matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
                    cropped_poly = cv2.warpPerspective(img, perspective_matrix, (height, width))

                des_path = os.path.join(dir, f'poly-{i}.jpg')
                if cropped_poly is not None:
                    cv2.imwrite(des_path, cropped_poly)
                else:
                    pass
    print(f"Number of text instances: {instances}")
    print(f"Number of images without text: {no_text} ({(100 * no_text/num_file):.2f}%)")

In [ ]:
# modified for panoramic data, to keep tree structure
def extract_text_p(origin, destination):
    """
    :origin: Folder containing images, grouped by gentrified/non-gentrified
    :destination: Folder to contain cropped-out text instances, grouped by gentrified/non-gentrified
    """
    no_text = 0
    num_file = 0
    instances = 0
    record = os.path.join(destination, "img-with-text.txt")
    open(record, 'a').close()

    # neighborhood per class (gentrified/non-gentrified)
    nbhd_dir = glob.glob(os.path.join(origin,"*")) # AmsPano/images_classified/{(non)gentrified}/{* = nbhd}
    for nbhd_folder in nbhd_dir:
        nbhd_name = os.path.split(nbhd_folder)[1]
        des_nbhd_folder = os.path.join(destination, nbhd_name) # AmsPano/images_cropped/{(non)gentrified}/{nbhd}
        os.makedirs(des_nbhd_folder) # Make corresponding folder for nbhd in new directory

        # location per nbhd (lon-lat)
        location_dir = glob.glob(os.path.join(nbhd_folder, "*"))
        for location in location_dir:
            location_name = os.path.split(location)[1]
            des_location_folder = os.path.join(des_nbhd_folder, location_name) # AmsPano/images_cropped/{(non)gentrified}/{* = nbhd}

            # img per location
            img_dir = glob.glob(os.path.join(location, "*.jpg"))
            for image in img_dir:
                num_file += 1
                img_name = os.path.split(image)[1]

                # Read img
                img = cv2.imread(image)
                # Detect texts
                result = reader.detect(img, add_margin=0)

                # Crop & save text regions
                if not any([result[0][0], result[1][0]]): # If no text detected
                    no_text += 1
                else: # If text detected
                    with open(str(record), "a") as f:
                        f.write(img_name + '\n') # Record img name
                    if not os.path.exists(des_location_folder):
                        os.makedirs(des_location_folder) # Make corresponding folder for img location in new directory

                    for i, box in enumerate(result[0][0]):
                        instances += 1
                        tl = (max(0,box[0]), max(0,box[2]))
                        br = (max(0,box[1]), max(0,box[3]))
                        cropped_rec = img[tl[1]:br[1], tl[0]:br[0]]

                        des_path = os.path.join(des_location_folder, f'rec-{i}.jpg')
                        if cropped_rec is not None:
                            cv2.imwrite(des_path, cropped_rec)
                        else:
                            pass

                    for i, box in enumerate(result[1][0]):
                        instances += 1
                        rect = cv2.minAreaRect(np.array(box, dtype=np.int32))

                        width = int(rect[1][0])
                        height = int(rect[1][1])

                        bound = cv2.boxPoints(rect) ; bound = np.int0(bound)

                        src_pts = bound.astype("float32")

                        if width > height:
                            dst_pts = np.array([[0, height-1],
                                                [0, 0],
                                                [width-1, 0],
                                                [width-1, height-1]],
                                                dtype="float32")
                            perspective_matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
                            cropped_poly = cv2.warpPerspective(img, perspective_matrix, (width, height))
                        else:
                            dst_pts = np.array([[0, 0],
                                                [height-1, 0],
                                                [height-1, width-1],
                                                [0, width-1]],
                                                dtype="float32")

                            perspective_matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
                            cropped_poly = cv2.warpPerspective(img, perspective_matrix, (height, width))

                        des_path = os.path.join(des_location_folder, f'poly-{i}.jpg')
                        if cropped_poly is not None:
                            cv2.imwrite(des_path, cropped_poly)
                        else:
                            pass
    print(f"Number of text instances: {instances}")
    print(f"Number of images without text: {no_text} ({(100 * no_text/num_file):.2f}%)")

In [ ]:
# modified for panoramic data LEFT & RIGHT views, keeping tree structure
def extract_text_lr(origin, destination):
    """
    :origin: Folder containing images, grouped by gentrified/non-gentrified
    :destination: Folder to contain cropped-out text instances, grouped by gentrified/non-gentrified
    """
    no_text = 0
    num_file = 0
    instances = 0
    record = os.path.join(destination, "lr-img-with-text.txt")
    open(record, 'a').close()

    # neighborhood per class (gentrified/non-gentrified)
    nbhd_dir = glob.glob(os.path.join(origin,"*")) # AmsPano/images_classified/{(non)gentrified}/{* = nbhd}
    for nbhd_folder in nbhd_dir:
        nbhd_name = os.path.split(nbhd_folder)[1]
        des_nbhd_folder = os.path.join(destination, nbhd_name) # AmsPano/images_cropped/{(non)gentrified}/{nbhd}
        os.makedirs(des_nbhd_folder) # Make corresponding folder for nbhd in new directory

        # location per nbhd (lon-lat)
        location_dir = glob.glob(os.path.join(nbhd_folder, "*"))
        for location in location_dir:
            location_name = os.path.split(location)[1]
            des_location_folder = os.path.join(des_nbhd_folder, location_name) # AmsPano/images_cropped/{(non)gentrified}/{* = nbhd}

            # img in subfolder per location
            img_dir = glob.glob(os.path.join(location, "*/*.jpg"))
            for image in img_dir:
                img_name = os.path.split(image)[1]
                if img_name == "_left.jpg" or img_name == "_right.jpg":
                    num_file += 1
                    # Read img
                    img = cv2.imread(image)
                    # Detect texts
                    result = reader.detect(img, add_margin=0)

                    # Crop & save text regions
                    if not any([result[0][0], result[1][0]]): # If no text detected
                        no_text += 1
                    else: # If text detected
                        with open(str(record), "a") as f:
                            f.write(location_name + '\n') # Record img name
                        if not os.path.exists(des_location_folder):
                            os.makedirs(des_location_folder) # Make corresponding folder for img location in new directory

                        for i, box in enumerate(result[0][0]):
                            instances += 1
                            tl = (max(0,box[0]), max(0,box[2]))
                            br = (max(0,box[1]), max(0,box[3]))
                            cropped_rec = img[tl[1]:br[1], tl[0]:br[0]]

                            des_path = os.path.join(des_location_folder, f'rec-{i}.jpg')
                            if cropped_rec is not None:
                                cv2.imwrite(des_path, cropped_rec)

                        for i, box in enumerate(result[1][0]):
                            instances += 1
                            rect = cv2.minAreaRect(np.array(box, dtype=np.int32))

                            width = int(rect[1][0])
                            height = int(rect[1][1])

                            bound = cv2.boxPoints(rect) ; bound = np.int0(bound)

                            src_pts = bound.astype("float32")

                            if width > height:
                                dst_pts = np.array([[0, height-1],
                                                    [0, 0],
                                                    [width-1, 0],
                                                    [width-1, height-1]],
                                                    dtype="float32")
                                perspective_matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
                                cropped_poly = cv2.warpPerspective(img, perspective_matrix, (width, height))
                            else:
                                dst_pts = np.array([[0, 0],
                                                    [height-1, 0],
                                                    [height-1, width-1],
                                                    [0, width-1]],
                                                    dtype="float32")

                                perspective_matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
                                cropped_poly = cv2.warpPerspective(img, perspective_matrix, (height, width))

                            des_path = os.path.join(des_location_folder, f'poly-{i}.jpg')
                            if cropped_poly is not None:
                                cv2.imwrite(des_path, cropped_poly)

    print(f"Number of text instances: {instances}")
    print(f"Number of images without text: {no_text} ({(100 * no_text/num_file):.2f}%)")

In [ ]:
# modified for panoramic data queried w/ OSM, keeping tree structure
def extract_text_osm(origin, destination):
    """
    :origin: Folder containing images, grouped by gentrified/non-gentrified
    :destination: Folder to contain cropped-out text instances, grouped by gentrified/non-gentrified
    """
    no_text = 0
    num_file = 0
    instances = 0
    record = os.path.join(destination, "osm-img-with-text.txt")
    open(record, 'a').close()

    # location per nbhd (lon-lat)
    location_dir = glob.glob(os.path.join(origin, "*"))
    for location in location_dir:
        location_name = os.path.split(location)[1]
        des_location_folder = os.path.join(destination, location_name) # AmsPano/images_cropped/{(non)gentrified}/{* = nbhd}

        # img in subfolder per location
        img_dir = glob.glob(os.path.join(location, "*.jpg"))
        for image in img_dir:
            img_name = os.path.split(image)[1]
            num_file += 1
            # Read img
            img = cv2.imread(image)
            # Detect texts
            result = reader.detect(img, add_margin=0)

            # Crop & save text regions
            if not any([result[0][0], result[1][0]]): # If no text detected
                no_text += 1
            else: # If text detected
                with open(str(record), "a") as f:
                    f.write(location_name + '/' + image_name + '\n') # Record img
                if not os.path.exists(des_location_folder):
                    os.makedirs(des_location_folder) # Make corresponding folder for img location in new directory

                for i, box in enumerate(result[0][0]):
                    instances += 1
                    tl = (max(0,box[0]), max(0,box[2]))
                    br = (max(0,box[1]), max(0,box[3]))
                    cropped_rec = img[tl[1]:br[1], tl[0]:br[0]]

                    des_path = os.path.join(des_location_folder, f'rec-{i}.jpg')
                    if cropped_rec is not None:
                        cv2.imwrite(des_path, cropped_rec)

                for i, box in enumerate(result[1][0]):
                    instances += 1
                    rect = cv2.minAreaRect(np.array(box, dtype=np.int32))

                    width = int(rect[1][0])
                    height = int(rect[1][1])

                    bound = cv2.boxPoints(rect) ; bound = np.int0(bound)

                    src_pts = bound.astype("float32")

                    if width > height:
                        dst_pts = np.array([[0, height-1],
                                            [0, 0],
                                            [width-1, 0],
                                            [width-1, height-1]],
                                            dtype="float32")
                        perspective_matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
                        cropped_poly = cv2.warpPerspective(img, perspective_matrix, (width, height))
                    else:
                        dst_pts = np.array([[0, 0],
                                            [height-1, 0],
                                            [height-1, width-1],
                                            [0, width-1]],
                                            dtype="float32")

                        perspective_matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
                        cropped_poly = cv2.warpPerspective(img, perspective_matrix, (height, width))

                    des_path = os.path.join(des_location_folder, f'poly-{i}.jpg')
                    if cropped_poly is not None:
                        cv2.imwrite(des_path, cropped_poly)

    print(f"Number of text instances: {instances}")
    print(f"Number of images without text: {no_text} ({(100 * no_text/num_file):.2f}%)")

# StreetSwipe

In [ ]:
# make folders for cropped-out texts
# class_names = ['gentrified','non-gentrified']
# new_folder = '/content/drive/MyDrive/MASTERS/Thesis/Data/StreetSwipe/images_cropped'
# for name in class_names:
#     os.makedirs(os.path.join(new_folder, name))

In [ ]:
origin_gen = '/content/drive/MyDrive/MASTERS/Thesis/Data/StreetSwipe/images_classified/gentrified/'
origin_nongen = '/content/drive/MyDrive/MASTERS/Thesis/Data/StreetSwipe/images_classified/non-gentrified/'

### Gentrified

In [ ]:
destination_gen = '/content/drive/MyDrive/MASTERS/Thesis/Data/StreetSwipe/images_cropped/gentrified/'
# extract_text(origin_gen, destination_gen)

Number of text instances: 2819
Number of images without text: 24 (3.38%)


### Non-gentrified

In [ ]:
destination_nongen = '/content/drive/MyDrive/MASTERS/Thesis/Data/StreetSwipe/images_cropped/non-gentrified/'
# extract_text(origin_nongen, destination_nongen)

Number of text instances: 7680
Number of images without text: 109 (6.30%)


# Panoramic left&right

In [ ]:
# make folders for cropped-out texts
# class_names = ['gentrified','non-gentrified']
# new_folder = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_cropped_lr'
# for name in class_names:
#     os.makedirs(os.path.join(new_folder, name))

### Gentrified

In [ ]:
origin_gen_lr = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_classified/gentrified/'
destination_gen_lr = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_cropped_lr/gentrified/'
# extract_text_lr(origin_gen_lr, destination_gen_lr)

Number of text instances: 1738
Number of images without text: 3658 (80.04%)


### Non-gentrified

In [ ]:
origin_nongen_lr = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_classified/non-gentrified/'
destination_nongen_lr = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_cropped_lr/non-gentrified/'
# extract_text_lr(origin_nongen_lr, destination_nongen_lr)

Number of text instances: 1139
Number of images without text: 3720 (85.40%)


# OSM

In [ ]:
# make folders for cropped-out texts
# class_names = ['gentrified','non-gentrified']
# new_folder = '/content/drive/MyDrive/MASTERS/Thesis/Data/OSM/osm_1/cropped/'
# for name in class_names:
#     os.makedirs(os.path.join(new_folder, name))

### Gentrified

In [ ]:
origin_gen_osm = '/content/drive/MyDrive/MASTERS/Thesis/Data/OSM/osm_1/classified/gentrified/' # 402
destination_gen_osm = '/content/drive/MyDrive/MASTERS/Thesis/Data/OSM/osm_1/cropped/gentrified/'
extract_text_osm(origin_gen_osm, destination_gen_osm)

Number of text instances: 477
Number of images without text: 550 (68.41%)


### Non-gentrified

In [ ]:
origin_nongen_osm = '/content/drive/MyDrive/MASTERS/Thesis/Data/OSM/osm_1/classified/non-gentrified/' # 96
destination_nongen_osm = '/content/drive/MyDrive/MASTERS/Thesis/Data/OSM/osm_1/cropped/non-gentrified/'
extract_text_osm(origin_nongen_osm, destination_nongen_osm)

Number of text instances: 241
Number of images without text: 98 (51.04%)


# Panoramas

In [ ]:
# make folders for cropped-out texts
# class_names = ['gentrified','non-gentrified']
# new_folder = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_cropped'
# for name in class_names:
#     os.makedirs(os.path.join(new_folder, name))

### Gentrified

In [ ]:
origin_gen_p = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_classified/gentrified/'
destination_gen_p = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_cropped/gentrified/'
# extract_text_p(origin_gen_p, destination_gen_p)

Number of text instances: 5132
Number of images without text: 831 (36.37%)


### Non-gentrified

In [ ]:
origin_nongen_p = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_classified/non-gentrified/'
destination_nongen_p = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_cropped/non-gentrified/'
# extract_text_p(origin_nongen_p, destination_nongen_p)

Number of text instances: 3386
Number of images without text: 814 (48.66%)


# Panoramic left&right&front

In [ ]:
# make folders for cropped-out texts
# class_names = ['gentrified','non-gentrified']
# new_folder = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_cropped_lrf'
# for name in class_names:
#     os.makedirs(os.path.join(new_folder, name))

### Gentrified

In [ ]:
origin_gen_lrf = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_classified/gentrified/'
destination_gen_lrf = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_cropped_lrf/gentrified/'
# extract_text_lrf(origin_gen_lrf, destination_gen_lrf)

Number of text instances: 2113
Number of images without text: 5691 (83.02%)


### Non-gentrified

In [ ]:
origin_nongen_lrf = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_classified/non-gentrified/'
destination_nongen_lrf = '/content/drive/MyDrive/MASTERS/Thesis/Data/AmsPano/images_cropped_lrf/non-gentrified/'
# extract_text_lrf(origin_nongen_lrf, destination_nongen_lrf)

Number of text instances: 1418
Number of images without text: 5707 (87.34%)
